# Importing packages; setting samplers

importing and setting dwave-2000q machine

In [1]:
import numpy as np
import dimod
import matplotlib.pyplot as plt
import dwave_networkx as dnx
import networkx as nx
import minorminer
import minorminer.layout as mml
%matplotlib inline
import dwave.inspector
import matplotlib as mpl
from datetime import datetime
from pathlib import Path  
import pandas as pd

In [2]:

from dwave.system import DWaveSampler

# Use a D-Wave system as the sampler
f = open("token.txt", "r")
#sampler = DWaveSampler(solver=dict(topology__type='chimera'),token=f.read())
sampler = DWaveSampler(solver=dict(topology__type='chimera'),token=f.read())

print("QPU {} was selected.".format(sampler.solver.name))

QPU DW_2000Q_6 was selected.


setting simulated annealing sampler

# Unit cell

In [3]:
#setting the unit cell chains. It contains 32 qubits and 16 sites
#so each site is a 2chain
#first row has qubits, second row has the site that each qubit is in
uc_qubits=list(range(0,16))+list(range(128,128+16))
uc_sites=[
    4,0,5,1,0,1,4,5,
    2,6,3,7,3,2,7,6,
    8,12,9,13,9,8,13,12,
    14,10,15,11,10,11,14,15
]

uc_qubits_sites=np.array([
    uc_qubits,
    uc_sites
])

In [4]:
print(uc_qubits_sites)

[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15 128 129
  130 131 132 133 134 135 136 137 138 139 140 141 142 143]
 [  4   0   5   1   0   1   4   5   2   6   3   7   3   2   7   6   8  12
    9  13   9   8  13  12  14  10  15  11  10  11  14  15]]


In [5]:
#setting the couplings in the unit cell
#there are 24 couplings
uc_coups_tups=[
    #intra-star couplings
    (0,4),    (1,5),    (2,6),    (3,7),
    (11,12),  (10,13),  (9,14),   (8,15),
    (131,132),(130,133),(129,134),(128,135),
    (136,140),(137,141),(138,142),(139,143),
    #inter-star horizontal couplings
    (5,13),(7,15),(132,140),(134,142),
    #inter-star vertical couplings
    (0,128),(2,130),(9,137),(11,139)
]

#turning the tuples into an array
uc_coups_qubits=[[],[]]
for i in range(len(uc_coups_tups)):
    uc_coups_qubits[0].append(uc_coups_tups[i][0])
    uc_coups_qubits[1].append(uc_coups_tups[i][1])
uc_coups_qubits=np.array(uc_coups_qubits)

In [6]:
#setting the x and y tiling couplings (that connect unit cells)
uc_tilx_tups=[
    (12,20),(14,22),(141,149),(143,151)
]
uc_tily_tups=[
    (128,128+128),(131,131+128),(136,136+128),(138,138+128)
]

#turning the tuples into arrays
uc_tilx=[[],[]]
uc_tily=[[],[]]

for i in range(len(uc_tilx_tups)):
    uc_tilx[0].append(uc_tilx_tups[i][0])
    uc_tilx[1].append(uc_tilx_tups[i][1])
for i in range(len(uc_tily_tups)):    
    uc_tily[0].append(uc_tily_tups[i][0])
    uc_tily[1].append(uc_tily_tups[i][1])
    
uc_tilx=np.array(uc_tilx)
uc_tily=np.array(uc_tily)

# Functions for: tiling, getting coupling dictionaries

In [7]:
#these functions can take any array and tile it right or down
def tile_right(arr_in):
    add_arr=16*np.ones(arr_in.shape)
    arr_out=arr_in+add_arr
    return arr_out

def tile_down(arr_in):
    add_arr=256*np.ones(arr_in.shape)
    arr_out=arr_in+add_arr
    return arr_out

In [8]:
def NxN_tiling(N):
    #initialize registries with the appropriate unit cell arrays
    ucreg=[]
    coupsreg=[]
    tilxreg=[]
    tilyreg=[]
    
    ucreg.append(uc_qubits_sites)
    coupsreg.append(uc_coups_qubits)
    tilxreg.append(uc_tilx)
    tilyreg.append(uc_tily)

    #tile the unit cell N-1 times to the right
    #tile the unit cell couplings N-1 times to the right
    #tile the tilingy couplings N-1 times to the right
    #store everything in the registries
    for i in range(N-1):
        ucreg.append(tile_right(ucreg[i]))
        coupsreg.append(tile_right(coupsreg[i]))
        tilyreg.append(tile_right(tilyreg[i]))
    #tile the tilingx couplings N-2 times to the right
    for i in range(N-2):
        tilxreg.append(tile_right(tilxreg[i]))
        
    #now every registry has the arrays for the first row of unit cells
    ucrow=ucreg
    coupsrow=coupsreg
    tilxrow=tilxreg
    tilyrow=tilyreg
    
    #tile down each unit cell in the first row N-1 times
    #tile down each unit cell couplings in the first row N-1 times
    #tile down each tilingx in the first row N-1 times
    for i in range(N-1):
        newrowuc=[]
        for j in range(len(ucrow)):
            ucreg.append(tile_down(ucrow[j]))
            newrowuc.append(tile_down(ucrow[j]))
        ucrow=newrowuc
    for i in range(N-1):
        newrowcoups=[]
        for j in range(len(coupsrow)):
            coupsreg.append(tile_down(coupsrow[j]))
            newrowcoups.append(tile_down(coupsrow[j]))
        coupsrow=newrowcoups
    for i in range(N-1):
        newrowtilx=[]
        for j in range(len(tilxrow)):
            tilxreg.append(tile_down(tilxrow[j]))
            newrowtilx.append(tile_down(tilxrow[j]))
        tilxrow=newrowtilx
        
    #tile down each tilingy in the first row N-2 times
    for i in range(N-2):
        newrowtily=[]
        for j in range(len(tilyrow)):
            tilyreg.append(tile_down(tilyrow[j]))
            newrowtily.append(tile_down(tilyrow[j]))
        tilyrow=newrowtily
    
    if N==1:
        tilxreg=[]
        tilyreg=[]
    
    return ucreg,coupsreg,tilxreg,tilyreg

In [9]:
def get_coups(ucreg,coupsreg,tilxreg,tilyreg,jchain,jcoup):
 
    #get a list of all 2chains as tuples
    #the tuples will take the form (site,(qubit1,qubit2))
    sites_2chains=[]
    for i in range(len(ucreg)):
        ucarr=ucreg[i]
        qubits=ucarr[0]
        sites=ucarr[1]
        uc_sites_2chains=[]
        for j in range(len(sites)):
            for k in range(len(sites)):
                if sites[j]==sites[k]:
                    site=sites[k]
                    qubit1=qubits[j]
                    qubit2=qubits[k]
                    if qubit1!=qubit2:
                        list_2chain=[qubit1,qubit2]
                        list_2chain.sort()
                        uc_sites_2chains.append((site,tuple(list_2chain)))   
        uc_sites_2chains=list(set(uc_sites_2chains))
        sites_2chains.append(uc_sites_2chains)

    #get a list of couplings as tuples
    coups_tups=[]
    for i in range(len(coupsreg)):
        uc_coups_tups=[]
        coupsarr=coupsreg[i]
        for j in range(len(coupsarr[0])):
            qubit1=coupsarr[0][j]
            qubit2=coupsarr[1][j]
            uc_coups_tups.append((qubit1,qubit2))
        coups_tups.append(uc_coups_tups)

    #get a list of tilx couplings as tuples
    tilx_tups=[]
    for i in range(len(tilxreg)):
        uc_tilx_tups=[]
        tilxarr=tilxreg[i]
        for j in range(len(tilxarr[0])):
            qubit1=tilxarr[0][j]
            qubit2=tilxarr[1][j]
            uc_tilx_tups.append((qubit1,qubit2))
        tilx_tups.append(uc_tilx_tups)

    #get a list of tily couplings as tuples
    tily_tups=[]
    for i in range(len(tilyreg)):
        uc_tily_tups=[]
        tilyarr=tilyreg[i]
        for j in range(len(tilyarr[0])):
            qubit1=tilyarr[0][j]
            qubit2=tilyarr[1][j]
            uc_tily_tups.append((qubit1,qubit2))
        tily_tups.append(uc_tily_tups)


    #get a list of 2chains as dictionaries
    dict_2chains={}
    for i in range(len(sites_2chains)):
        for j in range(len(sites_2chains[0])):
            qubit_tuple=sites_2chains[i][j][1]
            dict_2chains.update({qubit_tuple:jchain})

    #get a list of couplings as dictionaries
    dict_coups={}
    for i in range(len(coups_tups)):
        for j in range(len(coups_tups[0])):
            qubit_tuple=coups_tups[i][j]
            dict_coups.update({qubit_tuple:jcoup})

    #get a list of tilx couplings as dictionaries
    dict_tilx={}
    for i in range(len(tilx_tups)):
        for j in range(len(tilx_tups[0])):
            qubit_tuple=tilx_tups[i][j]
            dict_tilx.update({qubit_tuple:jcoup})

    #get a list of tily couplings as dictionaries
    dict_tily={}
    for i in range(len(tily_tups)):
        for j in range(len(tily_tups[0])):
            qubit_tuple=tily_tups[i][j]
            dict_tily.update({qubit_tuple:jcoup})
    
    print(len(dict_2chains),"2chains")
    print(len(dict_coups),"inter-site couplings")
    print(len(dict_tilx),"x tiling couplings")
    print(len(dict_tily),"y tiling couplings")

    allcouplings_dict={}
    allcouplings_dict.update(dict_2chains)
    allcouplings_dict.update(dict_coups)
    allcouplings_dict.update(dict_tilx)
    allcouplings_dict.update(dict_tily)
    
    print(len(allcouplings_dict),"total couplings")

    return allcouplings_dict,sites_2chains

# Functions for: checking all qubits, deleting sites with missing qubits

In [10]:
#checking nodes and making a list of missing ones
import termcolor
from termcolor import colored

def check_nodes(nodlist):
    missingqs=[]
    if all(qubit in sampler.nodelist for qubit in nodlist)==True:
        print("all nodes are good")
    else:
        for node in nodlist:
            if node not in sampler.nodelist:
                missingqs.append(node)
                print(colored(node, 'red'), colored('is not found in list', 'red'))
    return missingqs

In [11]:
def split_oldcouplers(oldcoups,oldtups):    
    oldcoupslist=[]
    old2chainslist=[]

    for i in range(len(oldtups)):
        tupl=oldtups[i]
        if (oldcoups[tupl]==jcoup):
            oldcoupslist.append(tupl)
        if oldcoups[tupl]==j2chain:
            old2chainslist.append(tupl)

    return oldcoupslist,old2chainslist

In [12]:
def to_remove(missingqs,oldcoupslist,old2chainslist):
    #make lists of all 2chains, and coups to remove
    remove2chains=[]
    removecoups=[]
    
    for i in range(len(missingqs)):
        missq = missingqs[i]

        #for each missq, identify what 2chain it's in
        for i, tup2chain in enumerate(old2chainslist):
            if (tup2chain[0] == missq or tup2chain[1] == missq):
                chain=tup2chain
                remove2chains.append(chain)

        #identify all couplings associated with the chain
        for qubit in chain:
            for i, tupcoup in enumerate(oldcoupslist):
                if (tupcoup[0] == qubit or tupcoup[1] == qubit):
                    removecoups.append(tupcoup)
    
    
    #remove repeats in remove2chains
    remove2chainsnew=[]
    remove2chains=set(remove2chains)
    for tup in remove2chains:
        remove2chainsnew.append(tup)
    remove2chains=remove2chainsnew
    
    #remove repeats in removecoups
    removecoupsnew=[]
    removecoups=set(removecoups)
    for tup in removecoups:
        removecoupsnew.append(tup)
    removecoups=removecoupsnew


    #combining the remove lists
    allremove=[]
    for tup in remove2chains:
        allremove.append(tup)
    for tup in removecoups:
        allremove.append(tup)
    
    return allremove

In [13]:
def Reverse(tuples):
    new_tup = tuples[::-1]
    return new_tup

In [14]:
def remove_couplers(couplerlist,oldtups,allremove):
    #removing the couplers from the master couplers list
    print(len(couplerlist), "couplers before")
    for tup in allremove:
        if tup in oldtups:
            print(tup, "is good, removing from couplers")
            couplerlist.pop(tup)
        else:
            print(tup,"is bad")
            if Reverse(tup) in oldtups:
                print(Reverse(tup),"is good, removing from couplers")
                couplerlist.pop(Reverse(tup))
            else:
                print("what the")

    #extracting tuples from coupler dictionary
    newtuples_only=[]
    for edge in couplerlist.keys():
        newtuples_only.append(edge)

    print(len(couplerlist), "couplers after")
    
    #recheck tuples
    if all(edge in sampler.edgelist for edge in newtuples_only)==True:
        print("all couplers are good")
    
    return couplerlist

In [15]:
#set_anneal_schedule(tf,pause_time,pause_length)

# Setting N and J, running tiling functions, getting couplers, annealing

In [16]:
Num=7
j2chain=-2.0
jcoup=-1.0

regs=NxN_tiling(Num)

tempocouplers=get_coups(regs[0],regs[1],regs[2],regs[3],j2chain,jcoup)[0]
#to be used later
sites_2chains=get_coups(regs[0],regs[1],regs[2],regs[3],j2chain,jcoup)[1]

784 2chains
1176 inter-site couplings
168 x tiling couplings
168 y tiling couplings
2296 total couplings
784 2chains
1176 inter-site couplings
168 x tiling couplings
168 y tiling couplings
2296 total couplings


In [17]:
tempotuples=[]
for edge in tempocouplers.keys():
    tempotuples.append(edge)

nodes=[]
for i in tempotuples:
    val0=i[0]
    val1=i[1]
    nodes.append(val0)
    nodes.append(val1)
nodes=[*set(nodes)]
nodes.sort()
print("there are", len(nodes), "nodes")
print("")

missingqubits=check_nodes(nodes)
print("")

oldcoups=split_oldcouplers(tempocouplers,tempotuples)[0]
old2chains=split_oldcouplers(tempocouplers,tempotuples)[1]

removelist=to_remove(missingqubits,oldcoups,old2chains)
print(len(removelist),"couplers to be removed")
print("")

newcouplers=remove_couplers(tempocouplers,tempotuples,removelist)

missingcouplers=[]
if all(coupler in sampler.edgelist for coupler in newcouplers.keys())==True:
    print("all couplers are good")
else:
    for coupler in newcouplers.keys():
        if coupler not in sampler.edgelist:
#             print(colored(coupler, 'green'), colored('is found in list', 'green'))
#         else:
            print(colored(coupler, 'red'), colored('is not found in list', 'red'))
            missingcouplers.append(coupler)

for coupler in missingcouplers:
    print("removing",coupler)
    newcouplers.pop(coupler)
    
newnodes=[]
for tup in newcouplers.keys():
    val0=i[0]
    val1=i[1]
    newnodes.append(val0)
    newnodes.append(val1)
newnodes=[*set(nodes)]
newnodes.sort()
print("there are", len(newnodes), "nodes")
print("")

there are 1568 nodes

43.0 is not found in list
46.0 is not found in list
524.0 is not found in list
548.0 is not found in list
1723.0 is not found in list
1735.0 is not found in list

23 couplers to be removed

2296 couplers before
(43.0, 46.0) is good, removing from couplers
(1729.0, 1735.0) is good, removing from couplers
(522.0, 524.0) is good, removing from couplers
(1723.0, 1725.0) is good, removing from couplers
(545.0, 548.0) is good, removing from couplers
(540.0, 548.0) is good, removing from couplers
(524.0, 532.0) is good, removing from couplers
(522.0, 525.0) is good, removing from couplers
(1723.0, 1727.0) is good, removing from couplers
(394.0, 522.0) is good, removing from couplers
(1595.0, 1723.0) is good, removing from couplers
(1721.0, 1725.0) is good, removing from couplers
(41.0, 46.0) is good, removing from couplers
(1725.0, 1733.0) is good, removing from couplers
(1728.0, 1735.0) is good, removing from couplers
(1727.0, 1735.0) is good, removing from couplers
(52

In [27]:
ann_sch = [[0.0,0.0],[20,1.0]]
#ann_sch = [[0.0,0.0],[30.0,0.3],[40,0.3],[41,1.0]]
response = sampler.sample_ising(h = {}, J=newcouplers,
                                            num_reads=200,
                                            anneal_schedule=ann_sch, num_spin_reversal_transforms = 5)
dwave.inspector.show(response)

Serving Inspector on http://127.0.0.1:18000/?problemId=e566074a-ce4a-4381-a855-8c4c340e2b89

'http://127.0.0.1:18000/?problemId=e566074a-ce4a-4381-a855-8c4c340e2b89'

# Simulated annealing

Simulated annealing

In [328]:
#a big list containing a tuple for each sstar value
#each tuple looks like (sstar,[list of Nreads avg magnetizations])
biglist_sstar_mags=[]

tempotuples=[]
for edge in tempocouplers.keys():
    tempotuples.append(edge)
print("there are",len(tempotuples),"couplers")

nodes=[]
for i in tempotuples:
    val0=i[0]
    val1=i[1]
    nodes.append(val0)
    nodes.append(val1)
nodes=[*set(nodes)]
nodes.sort()
print("there are", len(nodes), "nodes")
print("")

h={}
# smallh=0.006
# for node in newnodes:
#     h.update({node:smallh})
# print("applied smallh=",smallh,"to all qubits being used")

# #state initialization for reverse annealing
# init={}
# for node in newnodes:
#     init.update({int(node):1})
# print("initial state set")

# #anneal-pause-anneal
# ann_sch=[[0.0,0.0],[5.0,0.1],[95.0,0.1],[100.0,1.0]]

#anneal-pause-quench
sstar=0.1
counter=0
while sstar<0.7:
    ann_sch=[[0.0,0.0],[(1000*sstar),sstar],[(1000.0*sstar)+100.0,sstar],[(1000.0*sstar)+100.0+1.0-sstar,1.0]]

    simresp = simsampler.sample_ising(
        h, J=tempocouplers, 
        num_reads=1000,
        beta_schedule=ann_sch
    )
    

    meanspinlist=[]
    for i in range(len(simresp.record)):
        read=simresp.record[i]
        spinlist=read[0]
        energy=read[1]
        Nocc=read[2]
        meanspin=np.mean(spinlist)
        for j in range(Nocc):
            meanspinlist.append(meanspin)
    sstar_mag=(sstar,meanspinlist)
    biglist_sstar_mags.append(sstar_mag)
    
    print("finished iteration",counter,";sstar=",sstar)
    sstar+=0.001
    counter+=1
print("done")

there are 3008 couplers
there are 2048 nodes

finished iteration 0 ;sstar= 0.1
finished iteration 1 ;sstar= 0.101
finished iteration 2 ;sstar= 0.10200000000000001
finished iteration 3 ;sstar= 0.10300000000000001
finished iteration 4 ;sstar= 0.10400000000000001
finished iteration 5 ;sstar= 0.10500000000000001
finished iteration 6 ;sstar= 0.10600000000000001
finished iteration 7 ;sstar= 0.10700000000000001
finished iteration 8 ;sstar= 0.10800000000000001
finished iteration 9 ;sstar= 0.10900000000000001
finished iteration 10 ;sstar= 0.11000000000000001
finished iteration 11 ;sstar= 0.11100000000000002
finished iteration 12 ;sstar= 0.11200000000000002
finished iteration 13 ;sstar= 0.11300000000000002
finished iteration 14 ;sstar= 0.11400000000000002
finished iteration 15 ;sstar= 0.11500000000000002
finished iteration 16 ;sstar= 0.11600000000000002
finished iteration 17 ;sstar= 0.11700000000000002
finished iteration 18 ;sstar= 0.11800000000000002
finished iteration 19 ;sstar= 0.119000000000

finished iteration 164 ;sstar= 0.2640000000000001
finished iteration 165 ;sstar= 0.2650000000000001
finished iteration 166 ;sstar= 0.2660000000000001
finished iteration 167 ;sstar= 0.2670000000000001
finished iteration 168 ;sstar= 0.2680000000000001
finished iteration 169 ;sstar= 0.26900000000000013
finished iteration 170 ;sstar= 0.27000000000000013
finished iteration 171 ;sstar= 0.27100000000000013
finished iteration 172 ;sstar= 0.27200000000000013
finished iteration 173 ;sstar= 0.27300000000000013
finished iteration 174 ;sstar= 0.27400000000000013
finished iteration 175 ;sstar= 0.27500000000000013
finished iteration 176 ;sstar= 0.27600000000000013
finished iteration 177 ;sstar= 0.27700000000000014
finished iteration 178 ;sstar= 0.27800000000000014
finished iteration 179 ;sstar= 0.27900000000000014
finished iteration 180 ;sstar= 0.28000000000000014
finished iteration 181 ;sstar= 0.28100000000000014
finished iteration 182 ;sstar= 0.28200000000000014
finished iteration 183 ;sstar= 0.283

finished iteration 326 ;sstar= 0.42600000000000027
finished iteration 327 ;sstar= 0.42700000000000027
finished iteration 328 ;sstar= 0.42800000000000027
finished iteration 329 ;sstar= 0.42900000000000027
finished iteration 330 ;sstar= 0.43000000000000027
finished iteration 331 ;sstar= 0.43100000000000027
finished iteration 332 ;sstar= 0.4320000000000003
finished iteration 333 ;sstar= 0.4330000000000003
finished iteration 334 ;sstar= 0.4340000000000003
finished iteration 335 ;sstar= 0.4350000000000003
finished iteration 336 ;sstar= 0.4360000000000003
finished iteration 337 ;sstar= 0.4370000000000003
finished iteration 338 ;sstar= 0.4380000000000003
finished iteration 339 ;sstar= 0.4390000000000003
finished iteration 340 ;sstar= 0.4400000000000003
finished iteration 341 ;sstar= 0.4410000000000003
finished iteration 342 ;sstar= 0.4420000000000003
finished iteration 343 ;sstar= 0.4430000000000003
finished iteration 344 ;sstar= 0.4440000000000003
finished iteration 345 ;sstar= 0.44500000000

KeyboardInterrupt: 